# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [1]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.psi4 import Psi4
from ase.optimize import QuasiNewton
from ase import Atoms, units
from ase.io import write
from time import perf_counter
from platform import node
from pathlib import Path
import numpy as np
import shutil
import json
import os

Configuration

In [2]:
molecule_name = 'caffeine'
method = 'hf'
basis = 'def2-svpd'
threads = min(os.cpu_count(), 12)

Derived

In [3]:
run_name = f'{molecule_name}_{method}_{basis}'

## Load in Target Molecule
We have it in a JSON file from PubChem

In [4]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [5]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [6]:
calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')


  Memory set to   3.815 GiB by Python driver.
  Threads set to 12 by Python driver.


In [7]:
%%time
atoms.calc = calc
dyn = QuasiNewton(atoms)
dyn.run(fmax=0.01)


  Memory set to   3.815 GiB by Python driver.
  Threads set to 12 by Python driver.
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:25:31   -18390.794139        4.1661
BFGSLineSearch:    1[  2] 15:26:05   -18391.020663        1.9419
BFGSLineSearch:    2[  4] 15:26:41   -18391.134456        1.3043
BFGSLineSearch:    3[  6] 15:27:16   -18391.190769        1.0587
BFGSLineSearch:    4[  8] 15:27:51   -18391.215384        0.5746
BFGSLineSearch:    5[ 10] 15:28:27   -18391.226240        0.3911
BFGSLineSearch:    6[ 12] 15:29:02   -18391.233268        0.3498
BFGSLineSearch:    7[ 14] 15:29:37   -18391.239126        0.2511
BFGSLineSearch:    8[ 16] 15:30:12   -18391.244270        0.2340
BFGSLineSearch:    9[ 18] 15:30:46   -18391.247254        0.1998
BFGSLineSearch:   10[ 20] 15:31:21   -18391.249083        0.1269
BFGSLineSearch:   11[ 22] 15:31:56   -18391.249935        0.1215
BFGSLineSearch:   12[ 24] 15:32:31   -18391.251072        0.1065
BFGSL

True

Save the output file

In [8]:
out_dir = Path('data') / 'exact'
out_dir.mkdir(exist_ok=True)

In [9]:
write(out_dir / f'{run_name}.xyz', atoms)

## Compute the Hessian using ASE
ASE has a built-in method which uses finite displacements

In [10]:
if Path('vib').is_dir():
    shutil.rmtree('vib')

In [11]:
%%time
finite_diff_time = perf_counter()
vib = Vibrations(atoms)
vib.run()
finite_diff_time = perf_counter() - finite_diff_time

CPU times: user 4h 37min 32s, sys: 9min 54s, total: 4h 47min 27s
Wall time: 26min 18s


Save the vibration data

In [12]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}-ase.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [13]:
vib_data.get_zero_point_energy()

5.5067174465850215

## Repeat with Psi4's analytic derivatives
See if we get the same answer faster

In [ ]:
analytic_time = perf_counter()
calc.set_psi4(atoms)
hess = calc.psi4.hessian(f'{method}/{basis}')
analytic_time = perf_counter() - analytic_time

In [ ]:
analytic_hess = hess.to_array() * units.Hartree / units.Bohr / units.Bohr

Convert it to an ASE object and save

In [ ]:
vib_data = VibrationsData.from_2d(atoms, analytic_hess)

In [ ]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}-psi4.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [ ]:
vib_data.get_zero_point_energy()

Save the runtimes

In [ ]:
with (out_dir / f'{run_name}-times.json').open('w') as fp:
    json.dump({
        'hostname': node(),
        'finite-diff': finite_diff_time,
        'analytic': analytic_time,
    }, fp)